### Get sample's filename

In [2]:
import pandas as pd
import pickle

In [3]:
def get_executable_sample() -> dict:
    df = pd.read_csv("../C malware info/sample_info.csv")
    family_lst = df.family.unique()
    sample_dct = {}
    for key in family_lst:
        sample_dct[key] = []

    for idx, row in df.iterrows():
        if row.platform in ['X86_64','X86','ARM']:
            sample_dct[row.family].append(row.filename)
    return sample_dct

sample_dct = get_executable_sample()

# save dictionary
with open('sample_dct.pkl', 'wb') as outp:
    pickle.dump(sample_dct, outp, pickle.HIGHEST_PROTOCOL)

In [7]:
# load dictionary
with open('sample_dct.pkl', 'rb') as inp:
    sample_dct = pickle.load(inp)

In [5]:
sample_dct

['16b4093813e2923e9ee70b888f0d50f972ac607253b00f25e4be44993d263bd2.bin',
 '28443c0a9bfd8a12c12a2aad3cc97d2e8998a9d8825fcf3643d46012f18713f0.bin',
 '46389c117c5f41b60e10f965b3674b3b77189b504b0aeb5c2da67adf55a7129f.bin',
 '7d3855bb09f2f6111d6c71e06e1e6b06dd47b1dade49af0235b220966c2f5be3.bin']

### Build 
[solved] Must run in `.py` file, cant run in `.ipynb`. Don't know why.
- 注意 trace folder 和 csv 的家族名稱不同，一個有 `-all` 一個沒有，兩者要一致否則會出 node 都是 0。
- 跑到 xorddos 的 ./trace/Dofloo/xxx 噴 FileNotFoundError: [WinError 2] 系統找不到指定的檔案。
    - f477b05826c46ac8a482e85d89b00de0.bin (有時候又OK)

In [11]:
# ! pip install graphviz

  Using cached graphviz-0.20.1-py3-none-any.whl (47 kB)


In [11]:
from cc_module import build

for family_name in sample_dct:
    if family_name != 'Dofloo': # 只建這個 family 的圖
        continue
    print(f'building family {family_name}')
    for s in sample_dct[family_name]:
        print(f'building sample {s}')
        try:
            # 建立圖譜並存檔
            build(family_name, s)
        except FileNotFoundError:
            print('[Error] FileNotFoundError')
    # [build(family_name, s) for s in sample_dct[family_name]] #　一行搞定版

building family Dofloo
building sample 0046a78514a658b9b7a4e29e8581f85b.bin
Node before reduction:  36
36
Nodes after reduction:  36
building sample 04b790e3b0dd70389c516216f81c6e5d.bin
Node before reduction:  1
1
Nodes after reduction:  1
building sample 121ca5362f9bc3fff5258b757e4250e5.bin
Node before reduction:  35
35
Nodes after reduction:  35
building sample 26d45f2e2d360ff1707c734006878dcb.bin
Node before reduction:  35
35
Nodes after reduction:  35
building sample 392214344b6d802dc4da644be260277e.bin
Node before reduction:  36
36
Nodes after reduction:  36
building sample 44d98df44327179d1a11dce725d0fa4a.bin
Node before reduction:  36
36
Nodes after reduction:  36
building sample 52ff8e32c390389f5bf18e0f0e2d9ada.bin
Node before reduction:  25
25
Nodes after reduction:  25
building sample 54a11aacadee0ed46e76373cfb8ed296.bin
Node before reduction:  36
36
Nodes after reduction:  36
building sample 5700e4b1c6498619a0e2562321363bd0.bin
Node before reduction:  28
28
Nodes after reduc

### Build 並回傳 ASG 資料結構
可以從 class AttackGraph 取得圖譜上的節點， asg.graph.keys()。

In [1]:
# 跑單一個圖
from cc_module import build

asg, g = build('Dofloo-all', '9a37fcc7eab08d59532bc5c66390bc30.bin')
type(g)

path: ./trace/Dofloo-all/9a37fcc7eab08d59532bc5c66390bc30.bin
Node before reduction:  36
36
Nodes after reduction:  36


graphviz.graphs.Digraph

In [8]:
names = [node.name for node in asg.graph.keys()]
set(names)

{'/dev/urandom',
 '/etc/init.d/boot.local',
 '/etc/rc.d/rc.local',
 '/etc/rc.local',
 '/proc/net/dev',
 '/proc/stat',
 '/sys/devices/system/cpu/online',
 '1529',
 '1530',
 '1531',
 '1532',
 '183.131.83.38:50050',
 'eth0',
 'eth1',
 'eth2',
 'eth3',
 'eth4',
 'eth5',
 'eth6',
 'eth7',
 'eth8',
 'eth9',
 'malware',
 'sed',
 'sh',
 'uname'}

### Nodes 統計紀錄


取得所有樣本的 Node，進行比對。

In [9]:
from cc_module import build
df = pd.read_csv("../C malware info/sample_info.csv")

# key: family -> nodes, value: freq counts of the node
nodes_value_cnts = dict.fromkeys(df.family.unique(), {})
for inx, row in df.iterrows():
    value_cnts = {}
    family_name = row.family
    sample_name = row.filename
    asg, g = build(family_name, sample_name)
    nodes = set([node.name for node in asg.graph.keys()]) # nodes of the asg in this sample
    for n in nodes: # record the freq of this sample
        if isinstance(n, int): # if node is pid -> ignore
            continue
        try:
            nodes_value_cnts[family_name][n] += 1
        except KeyError:
            nodes_value_cnts[family_name][n] = 1
    

path: ./trace/Xorddos/07c070b717a23453a2b71c7582d6a928.bin
Node before reduction:  1097
1097
Nodes after reduction:  1097
path: ./trace/Xorddos/0aefb67c01a24d05351b093455203fa2.bin
Node before reduction:  992
992
Nodes after reduction:  992
path: ./trace/Xorddos/0bc90c333f08237475a08c7158aba345.bin
Node before reduction:  991
991
Nodes after reduction:  991
path: ./trace/Xorddos/127ee9c007774e8f1101fb40ea05b42e.bin
Node before reduction:  1091
1091
Nodes after reduction:  1091
path: ./trace/Xorddos/16eb5d018d45024462dd63de470a8575.bin
Node before reduction:  1106
1106
Nodes after reduction:  1106
path: ./trace/Xorddos/24dca1b2097ad6c2e5d8b03d3022f010.bin
Node before reduction:  981
981
Nodes after reduction:  981
path: ./trace/Xorddos/39b38284c6ca30d09481fcb5dc7197aa.bin
Node before reduction:  1104
1104
Nodes after reduction:  1104
path: ./trace/Xorddos/7354323ff72a6132359c5aa76bf9904a.bin
Node before reduction:  976
976
Nodes after reduction:  976
path: ./trace/Xorddos/75a00aad5b5064

In [14]:
nodes_value_cnts['Xorddos']

{'2227': 11,
 '1664': 11,
 '2292': 14,
 '/bin/xthptzgcux': 1,
 '2019': 12,
 '1643': 13,
 '/bin/klkilsljlk': 1,
 '/usr/bin/flelsojwgd': 1,
 '/bin/xctzursddd': 1,
 '/usr/bin/xctzursddd': 1,
 '2132': 15,
 '1700': 12,
 '2473': 14,
 '2146': 13,
 '1849': 9,
 '/proc/stat': 34,
 '/bin/qcqzqtexbk': 1,
 '/tmp/klkilsljlk': 1,
 '2087': 12,
 '/tmp/ngyyxxrtfo': 1,
 '2460': 12,
 '1960': 7,
 '1957': 13,
 '2422': 19,
 '1603': 17,
 '1980': 17,
 '2140': 10,
 '/etc/rc5.d/S90myjhfrxfoq': 1,
 '/bin/xtldeywmzr': 1,
 '1595': 12,
 '2258': 12,
 '/tmp/rukbpaxnng': 1,
 '1953': 14,
 '2128': 14,
 '/usr/bin/bzelxedjhi': 1,
 '1967': 12,
 '2475': 16,
 '2237': 16,
 '2263': 14,
 '/tmp/xtldeywmzr': 1,
 '/usr/bin/uzxvmujqvg': 1,
 '1655': 12,
 '1877': 13,
 '1732': 8,
 '1574': 10,
 '2095': 11,
 '1892': 16,
 '2212': 15,
 '1515': 8,
 '/usr/bin/luklhvooxu': 1,
 '2355': 15,
 '2269': 15,
 '2331': 13,
 '1706': 12,
 '/bin/jtjcoldqrl': 1,
 '1936': 17,
 '/usr/bin/sfhhjasjbb': 1,
 '1902': 12,
 '2492': 9,
 '1808': 7,
 'uname': 36,
 '/

In [30]:
# analysis the nodes_value_cnts
with open('./nodes_value_cnts_analysis.txt','w') as file:
    family_name_lst = nodes_value_cnts.keys()
    print(family_name_lst)
    for f in family_name_lst:
        file.write(f"# Family {f}\n")
        nodes = nodes_value_cnts[f]
        for n in list(nodes.keys()):
            try:
                int(n)
                nodes.pop(n)
            except:
                pass
        sorted_nodes = dict(sorted(nodes.items(), key=lambda item: item[1], reverse=True))
        for k,v in sorted_nodes.items():
            file.write(f'{k}: {v}\n')
            if v > 1:
                print(f'{k}: {v}')
        file.write('\n\n')

        # 以 sort 過的 dict 取代原有的
        nodes_value_cnts[f] = sorted_nodes

dict_keys(['Xorddos', 'Mirai', 'Dofloo', 'Tsunami'])
malware: 41
uname: 36
sh: 35
/proc/stat: 34
sed: 32
8.8.8.8:53: 25
/etc/crontab: 21
chkconfig: 21
update-rc.d: 21
systemctl: 21
103.25.9.228:53: 14
/proc/net/dev: 13
/var/run/gcc4.pid: 12
/lib/libudev4.so: 12
/etc/cron.hourly/gcc4.sh: 12
eth5: 12
eth9: 12
eth4: 12
eth2: 12
eth3: 12
/dev/urandom: 12
eth0: 12
eth8: 12
eth7: 12
eth1: 12
eth6: 12
/etc/rc.local: 11
/etc/rc.d/rc.local: 11
/etc/init.d/boot.local: 11
/sys/devices/system/cpu/online: 10
0.0.0.0:8005: 9
103.240.141.50:8005: 9
0.0.0.0:80: 9
/etc/cron.hourly/cron.sh: 9
66.102.253.30:8005: 9
/lib/udev/udev: 9
103.25.9.245:8005: 9
/proc/rs_dev: 9
/var/run/sftp.pid: 9
0.0.0.0:60000: 7
0.0.0.0:53: 7
0.0.0.0:8080: 7
0.0.0.0:2444: 5
183.60.202.2:2444: 5
Unknown: 3
/proc/cpuinfo: 3
pidof: 3
/bin/watchdog: 2
/dev/misc/watchdog: 2
/dev/watchdog0: 2
/etc/watchdog: 2
/dev/watchdog: 2
154.91.161.81:48080: 2
/etc/mtab: 2
sysinfo: 2
/proc/sys/kernel/version: 2
/etc/ld.so.cache: 2
nul: 2
/etc/n

In [4]:
edges = graph.edges
for nodes, edge in edges.items():
    print(nodes.name)

{(<cc_module.Node at 0x254503a3430>,
  'read'): <cc_module.Edge at 0x254503a3160>,
 (<cc_module.Node at 0x254503a31f0>,
  'clone'): <cc_module.Edge at 0x254503a32b0>,
 (<cc_module.Node at 0x254503a31f0>,
  'clone'): <cc_module.Edge at 0x254503a3100>,
 (<cc_module.Node at 0x254503a31f0>,
  'clone'): <cc_module.Edge at 0x254503a3250>,
 (<cc_module.Node at 0x254503a31f0>,
  'clone'): <cc_module.Edge at 0x254503a3310>,
 (<cc_module.Node at 0x254503a31f0>,
  'clone'): <cc_module.Edge at 0x254503a33a0>,
 (<cc_module.Node at 0x254503a31f0>,
  'clone'): <cc_module.Edge at 0x254503a3130>,
 (<cc_module.Node at 0x254503a31f0>,
  'clone'): <cc_module.Edge at 0x254503a33d0>,
 (<cc_module.Node at 0x254503a3370>,
  'exec'): <cc_module.Edge at 0x254503a3610>,
 (<cc_module.Node at 0x254503a3490>,
  'read'): <cc_module.Edge at 0x254503a3190>,
 (<cc_module.Node at 0x254503a34f0>,
  'write'): <cc_module.Edge at 0x254503a3220>,
 (<cc_module.Node at 0x254503a3340>,
  'exec'): <cc_module.Edge at 0x254503a35e